# Node2vec

Danes bomo

1. podatke predstavili kot graf,
2. izračunali vpetje vseh vozlišč,
3. uporabili vpetja vozlišč, ki pripadajo poslovnim obratom, za napovedovanje vrste obrata.

## 1. Pretvorba v graf

Uporabili bomo knjižnico `networkx` (ki jo naložimo z ukazom `pip install networkx`).
Pri pretvorbi v graf moramo ugotviti, kaj so vozlišča in povezave grafa.

Odgovoriti moramo na kar nekaj vprašanj, npr.

- Kako predstavimo vrstice iz tabele `business` (stolpci `business_id`, `stars`, `review_count`, `category`)?
  - Ali sploh smemo v graf vnesti informacijo o `category` (glede na dogajanje pri točki 2)?
  - Kako predstaviti vrednosti za `review_count`?
- Kako predstavimo vrstice iz tabele `attributes` (stolpci `business_id`, `name`, `value`)?
  - Tukaj je namreč pripadajoča relacija trojiška.
- Kako predstaviti vrstice iz tabele `hours` (stolpci `business_id`, `day`, `open time`, `close time`)?
   - Dve trojiški relaciji, urejenost po času

Če odgovorimo na vse to, bomo vedeli tudi, kako pretvoriti vrstice tabel `review` in `users`.

In [39]:
import os
from typing import List
import numpy as np
import pandas as pd
import networkx as nx
from node2vec import Node2Vec
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import KBinsDiscretizer


def pripona_datoteke(dim, dolzina, stevilo, p, q):
    return f"dim{dim}_dolzina{dolzina}_stevilo{stevilo}_p{p}_q{q}"

### Igranje z networkx

In [40]:
import networkx as nx

g = nx.Graph()
# naredi vozlisce, lahko mu dajemo se atributr (..., ime_atributa=vrednost_atributa, ...)
g.add_node('oseba Maja')
# ustvarjanje povezav
g.add_edge('oseba Maja', 'moz karel')

In [41]:
g.edges

EdgeView([('oseba Maja', 'moz karel')])

# 1. Pretvorba v graf

In [42]:
def diskretiziraj(podatki: pd.DataFrame, atributi: List[str]):
    """Diskretizira izbrane atribute, ki se nahajajo v podatkih."""
    pretvornik = KBinsDiscretizer(
        n_bins=5,
        encode="ordinal",
        strategy="quantile"
    )
    nove_vrednosti = pretvornik.fit_transform(podatki[atributi])
    for i, a in enumerate(atributi):
        print(i, a, podatki[a].unique(), set(nove_vrednosti[:, i]))
        podatki[a] = nove_vrednosti[:, i]

## 1. 1 Pretvorba tabele business

Večina dela je že narejenega. Popravi kodo tako, da bomo lahko vključili še kakšen numeričen atribut, ki je morda koristen.

In [43]:


def posodobi_z_business(graf: nx.Graph):
    business = pd.read_csv("yelp_business.txt", sep="\t")
    ok_atributi = [
        # nekoristno: full_address, city, state, open
        # zaenkrat neprimerno za graf: latitude, longitude, review_count
        # prevovedano: category
        "stars", "review_count"
    ]
    diskretiziraj(business, ["review_count"])
    for _, vrsta in business.iterrows():
        vozlisce_b = f"biznis_{vrsta['business_id']}"
        vozlisca_a = [f"biznis_atribut_{a}_{vrsta[a]}" for a in ok_atributi]
        povezave = [(vozlisce_b, voz_a) for voz_a in vozlisca_a]
        for od, do in povezave:
            graf.add_edge(od, do)

## 1.2 Pretvorba tabele attributes

In [44]:
def posodobi_z_attributes(graf: nx.Graph):
    """Trojiške relacije zahtevajo ustanovitev 'posebnih vozlišč'"""
    attributes = pd.read_csv("yelp_attributes.txt", sep="\t")
        
    for _, vrsta in attributes.iterrows():
        vozlisce_b = f"biznis_{vrsta['business_id']}"
        vozlisce_ime_att = f"atribut_{vrsta['name']}"
        vozlisce_ime_vrednost_att = f"atribut_vrednost_{vrsta['name']}_{vrsta['value']}"
        povezave = [
            (vozlisce_b, vozlisce_ime_att),
            (vozlisce_ime_att, vozlisce_ime_vrednost_att),
            (vozlisce_b, vozlisce_ime_vrednost_att)
        ]
        for od, do in povezave:
            graf.add_edge(od, do)
    

## 1.3 Pretvorba tabele hours

In [45]:
def posodobi_z_hours(graf: nx.Graph):
    hours = pd.read_csv("yelp_hours.txt", sep="\t")
    for _, vrsta in hours.iterrows():
        vozlisce_b = f"biznis_{vrsta['business_id']}"
        vozlisce_dan = f"dan_{vrsta['day']}"
        vozlisce_dan_odpri = f"dan_odpri_{vrsta['day']}_{vrsta['open time']}"
        vozlisce_dan_zapri = f"dan_zapri_{vrsta['day']}_{vrsta['close time']}"
        povezave = [
            (vozlisce_b, vozlisce_dan),
            (vozlisce_b, vozlisce_dan_odpri),
            (vozlisce_b, vozlisce_dan_zapri),
            (vozlisce_dan, vozlisce_dan_odpri),
            (vozlisce_dan, vozlisce_dan_zapri)
        ]
        for od, do in povezave:
            graf.add_edge(od, do)


## 1.4 Pretvorba tabele review

In [46]:
def posodobi_z_review(graf: nx.Graph):
    reviews = pd.read_csv("yelp_review.txt", sep="\t")
    pass

## 1.5 Pretvoraba tabele users

In [47]:
def posodobi_z_users(graf: nx.Graph):
    users = pd.read_csv("yelp_users.txt", sep="\t").fillna(-1.0)
    pass


In [48]:
def yelp_to_graph():
    graf = nx.Graph()
    posodobi_z_business(graf)
    posodobi_z_attributes(graf)
    posodobi_z_review(graf)
    posodobi_z_hours(graf)
    posodobi_z_users(graf)
    return graf

## 2. Izračun vpetij

Uporabili bomo knjižnico `node2vec` (ki jo naložimo z ukazom `pip install node2vec`).
Algoritem temelji na algoritmu `word2vec`, ki je implementiran v knjižnici `gensim`,
ki se bo naložila v sklopu nameščanja knjižnice `node2vec`.

Algoritem, kot je vidno spodaj, ima kar nekaj parametrov.

In [49]:
def pridobi_vlozitev(
        graf: nx.Graph,
        dim=32,
        dolzina_sprehoda=10,
        st_sprehodov=200,
        p=2,
        q=1
):
    podrobnosti = pripona_datoteke(dim, dolzina_sprehoda, st_sprehodov, p, q)
    datoteka_model = f"node2vec_{podrobnosti}.model"
    datoteka_vektorji = f"node2vec_{podrobnosti}.csv"
    if not os.path.exists(datoteka_model):
        # Precompute probabilities and generate walks
        node2vec = Node2Vec(
            graf,
            dimensions=dim,
            walk_length=dolzina_sprehoda,
            num_walks=st_sprehodov,
            p=p,
            q=q,
            workers=4
        )
        # Embed nodes
        model = node2vec.fit(window=3, min_count=1, batch_words=50)
        # Save for later use
        model.wv.save_word2vec_format(datoteka_vektorji)
        model.save(datoteka_model)
    else:
        model = Word2Vec.load(datoteka_model)
    # Look for most similar nodes
    vozlisca_za_analizo = [
        'atribut_Alcohol',
        'dan_Monday'
    ]
    for v in vozlisca_za_analizo:
        print(f"Vozlišču {v} so najbolj podobna")
        for sosed, podobnost in model.wv.most_similar(v):
            print(f"    {sosed} (podobnost: {podobnost:.3f})")

# 3. Naredimo tabelo za učenje

- Iz `.csv`-ja preberemo upodobitve
- Iz tabele `business` poberemo vrednosti ciljne spremenljivke

In [50]:
def naredi_tabelo(
        dim=32,
        dolzina_sprehoda=10,
        st_sprehodov=200,
        p=2,
        q=1
):
    businesses = pd.read_csv("yelp_business.txt", sep="\t")
    biznisi = {f"biznis_{b}": i for i, b in enumerate(businesses["business_id"])}
    category = np.array(businesses["category"])
    pripona = pripona_datoteke(dim, dolzina_sprehoda, st_sprehodov, p, q)
    datoteka_vektorjev = f"node2vec_{pripona}.csv"
    with open(datoteka_vektorjev) as f:
        n_rows = len(biznisi)
        _, n_col = map(int, f.readline().strip().split(" "))
        matrix = np.zeros((n_rows, n_col))
        for row in f:
            row = row.split(" ")
            if row[0] in biznisi:
                e = [float(x) for x in row[1:]]
                matrix[biznisi[row[0]]] = np.array(e)
    return matrix, category

### Naučimo se modelov

In [51]:
yelp_graf = yelp_to_graph()
print(f"Končne statistike: {len(yelp_graf.nodes)}, {len(yelp_graf.edges)}")
pridobi_vlozitev(yelp_graf)
xs, y = naredi_tabelo()
x0, x1, y0, y1 = train_test_split(
    xs,
    y,
    test_size=0.25,
    random_state=1234,
    stratify=y
)
rf = RandomForestClassifier()  # n_estimators=300, max_features=1.0)
rf.fit(x0, y0)
y_hat = rf.predict(x1)
print("Točnost:", accuracy_score(y1, y_hat))

0 review_count [ 281   11  171   17  494   35   28   71  375    8   47   38   26    4
   44    5   25   15   12   14   84   19   13    3   18   10   20    7
  280  173   21   63    6    9 1314 1062   39  707   53   22   30   31
  274   36  456 1110   24  445  436  275 1355   73   88   49 1085  523
   61  144   62   45  104  817  345  810  702  874   60  460   34  562
 1289  747 1809 1876 2004  297  962  370 4084 1215 1413 2419  980  531
  406  660 2404 1088  485 1497  462 1052 1046  755  114 2791  616 1071
 1338  448  745  412  218  578  110  832  720  503  398  187  637 2201
  560  820   69   23  234  256  642  533  124  905   66  788   16  105
  204  207  519   79   29  743  253   81  109  102  492  154   37   42
  889  878  508  288 1076  106  308 3655   40  407  658  454  319   83
  772   41   58  718  502 1132   33  147  705  833 1019  790  395  430
  383  216  703 1336 2682  837  694  386  646  995  611  361  380  100
  663  255  717  655   50 1759  251   48   64  824  321 1657  

Computing transition probabilities:   0%|          | 0/1240 [00:00<?, ?it/s]

Generating walks (CPU: 4): 100%|██████████| 50/50 [00:04<00:00, 10.79it/s]


Vozlišču atribut_Alcohol so najbolj podobna
    atribut_vrednost_touristy_false (podobnost: 0.993)
    atribut_trendy (podobnost: 0.992)
    atribut_casual (podobnost: 0.990)
    atribut_vrednost_divey_false (podobnost: 0.988)
    atribut_Has TV (podobnost: 0.988)
    atribut_classy (podobnost: 0.987)
    atribut_lunch (podobnost: 0.987)
    atribut_hipster (podobnost: 0.986)
    atribut_vrednost_dessert_false (podobnost: 0.986)
    atribut_divey (podobnost: 0.986)
Vozlišču dan_Monday so najbolj podobna
    dan_Tuesday (podobnost: 0.972)
    dan_Wednesday (podobnost: 0.968)
    dan_Thursday (podobnost: 0.941)
    dan_Friday (podobnost: 0.925)
    dan_Saturday (podobnost: 0.736)
    dan_zapri_Monday_13:00:00 (podobnost: 0.728)
    dan_odpri_Monday_14:00:00 (podobnost: 0.712)
    dan_zapri_Monday_16:30:00 (podobnost: 0.689)
    dan_zapri_Monday_12:30:00 (podobnost: 0.679)
    dan_odpri_Wednesday_14:00:00 (podobnost: 0.664)
Točnost: 0.8860759493670886
